In [82]:
!pip install bs4
!pip install html5lib
!pip install requests
!pip install psycopg2-binary

You should consider upgrading via the 'c:\users\leonp\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\leonp\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [ ]:

import requests
from bs4 import BeautifulSoup
import psycopg2
import time
import calendar
from IPython.display import clear_output
import string
from collections import defaultdict


In [ ]:
def getOffersUrlsFromPage(url):
    pageContent = requests.get(url).content

    soup = BeautifulSoup(pageContent, 'html5lib')

    for i in soup.find_all('a', attrs={'class': 'offer-title__link'}):
        if not i.has_attr('data-test'):
            yield i['href']

In [ ]:
eqOptions = ['ABS',
             'CD',
             'Centralny zamek',
             'Elektryczne szyby przednie',
             'Elektrycznie ustawiane lusterka',
             'Immobilizer',
             'Poduszka powietrzna kierowcy',
             'Poduszka powietrzna pasażera',
             'Radio fabryczne',
             'Wspomaganie kierownicy',
             'Alarm',
             'Alufelgi',
             'ASR (kontrola trakcji)',
             'Asystent parkowania',
             'Asystent pasa ruchu',
             'Bluetooth',
             'Czujnik deszczu',
             'Czujnik martwego pola',
             'Czujnik zmierzchu',
             'Czujniki parkowania przednie',
             'Czujniki parkowania tylne',
             'Dach panoramiczny',
             'Elektrochromatyczne lusterka boczne',
             'Elektrochromatyczne lusterko wsteczne',
             'Elektryczne szyby tylne',
             'Elektrycznie ustawiane fotele',
             'ESP (stabilizacja toru jazdy)',
             'Gniazdo AUX',
             'Gniazdo SD',
             'Gniazdo USB',
             'Hak',
             'HUD (wyświetlacz przezierny)',
             'Isofix',
             'Kamera cofania',
             'Klimatyzacja automatyczna',
             'Klimatyzacja czterostrefowa',
             'Klimatyzacja dwustrefowa',
             'Klimatyzacja manualna',
             'Komputer pokładowy',
             'Kurtyny powietrzne',
             'Łopatki zmiany biegów',
             'MP3',
             'Nawigacja GPS',
             'Odtwarzacz DVD',
             'Ogranicznik prędkości',
             'Ogrzewanie postojowe',
             'Podgrzewana przednia szyba',
             'Podgrzewane lusterka boczne',
             'Podgrzewane przednie siedzenia',
             'Podgrzewane tylne siedzenia',
             'Poduszka powietrzna chroniąca kolana',
             'Poduszki boczne przednie',
             'Poduszki boczne tylne',
             'Przyciemniane szyby',
             'Radio niefabryczne',
             'Regulowane zawieszenie',
             'Relingi dachowe',
             'System Start-Stop',
             'Szyberdach',
             'Światła do jazdy dziennej',
             'Światła LED',
             'Światła przeciwmgielne',
             'Światła Xenonowe',
             'Tapicerka skórzana',
             'Tapicerka welurowa',
             'Tempomat',
             'Tempomat aktywny',
             'Tuner TV',
             'Wielofunkcyjna kierownica',
             'Zmieniarka CD']

eqOptionsTable = ['ABS',
             'CD',
             'Centralny_zamek',
             'Elektryczne_szyby_przednie',
             'Elektrycznie_ustawiane_lusterka',
             'Immobilizer',
             'Poduszka_powietrzna_kierowcy',
             'Poduszka_powietrzna_pasażera',
             'Radio_fabryczne',
             'Wspomaganie_kierownicy',
             'Alarm',
             'Alufelgi',
             'ASR_kontrola_trakcji',
             'Asystent_parkowania',
             'Asystent_pasa_ruchu',
             'Bluetooth',
             'Czujnik_deszczu',
             'Czujnik_martwego_pola',
             'Czujnik_zmierzchu',
             'Czujniki_parkowania_przednie',
             'Czujniki_parkowania_tylne',
             'Dach_panoramiczny',
             'Elektrochromatyczne_lusterka_boczne',
             'Elektrochromatyczne_lusterko_wsteczne',
             'Elektryczne_szyby_tylne',
             'Elektrycznie_ustawiane_fotele',
             'ESP_stabilizacja_toru_jazdy',
             'Gniazdo_AUX',
             'Gniazdo_SD',
             'Gniazdo_USB',
             'Hak',
             'HUD_wyświetlacz_przezierny',
             'Isofix',
             'Kamera_cofania',
             'Klimatyzacja_automatyczna',
             'Klimatyzacja_czterostrefowa',
             'Klimatyzacja_dwustrefowa',
             'Klimatyzacja_manualna',
             'Komputer_pokładowy',
             'Kurtyny_powietrzne',
             'Łopatki_zmiany_biegów',
             'MP3',
             'Nawigacja_GPS',
             'Odtwarzacz_DVD',
             'Ogranicznik_prędkości',
             'Ogrzewanie_postojowe',
             'Podgrzewana_przednia_szyba',
             'Podgrzewane_lusterka_boczne',
             'Podgrzewane_przednie_siedzenia',
             'Podgrzewane_tylne_siedzenia',
             'Poduszka_powietrzna_chroniąca_kolana',
             'Poduszki_boczne_przednie',
             'Poduszki_boczne_tylne',
             'Przyciemniane_szyby',
             'Radio_niefabryczne',
             'Regulowane_zawieszenie',
             'Relingi_dachowe',
             'System_Start_Stop',
             'Szyberdach',
             'Światła_do_jazdy_dziennej',
             'Światła_LED',
             'Światła_przeciwmgielne',
             'Światła_Xenonowe',
             'Tapicerka_skórzana',
             'Tapicerka_welurowa',
             'Tempomat',
             'Tempomat_aktywny',
             'Tuner_TV',
             'Wielofunkcyjna_kierownica',
             'Zmieniarka_CD']

In [ ]:
def keepDigits(str):
    keep = defaultdict(type(None))
    keep.update({ord(c): c for c in string.digits})
    return str.translate(keep)

In [ ]:
def getDetailsFromOffer(soup):
    result = {}
    for parameter in soup.find_all('li', attrs={'class': 'offer-params__item'}):
        key = parameter.find('span').text.replace(' ', '_').replace('(', '').replace(')', '')
        try:
            value = parameter.find('a').text
        except AttributeError:
            try:
                value = parameter.find('div').text
            except AttributeError:
                continue

        result[key] = value.strip().replace("'", '')

    if 'Moc' in result:
        result['Moc'] = int(result['Moc'][:-3].replace(' ', ''))
    if 'Pojemność_skokowa':
        result['Pojemność_skokowa'] = int(result['Pojemność_skokowa'][:-4].replace(' ', ''))
    if 'Przebieg' in result:
        result['Przebieg'] = int(result['Przebieg'][:-3].replace(' ', ''))
    if 'Przebieg' in result:
        result['Liczba_drzwi'] = int(result['Liczba_drzwi'])
    if 'Rok_produkcji' in result:
        result['Rok_produkcji'] = int(result['Rok_produkcji'])
        
    if 'Spalanie_W_Mieście' in result:
        result['Spalanie_W_Mieście'] = float(result['Spalanie_W_Mieście'][:-8].replace(',', '.'))
    if 'Emisja_CO2' in result:
        result['Emisja_CO2'] = float(result['Emisja_CO2'][:-5].replace(',', '.'))
    if 'Spalanie_W_Cyklu_Mieszanym' in result:
        result['Spalanie_W_Cyklu_Mieszanym'] = float(result['Spalanie_W_Cyklu_Mieszanym'][:-8].replace(',', '.'))
    if 'Spalanie_Poza_Miastem' in result:
        result['Spalanie_Poza_Miastem'] = float(result['Spalanie_Poza_Miastem'][:-8].replace(',', '.'))
    if 'Liczba_miejsc' in result:
        result['Liczba_miejsc'] = int(result['Liczba_miejsc'])
    if 'lub_do_przebieg_km' in result:
        result['lub_do_przebieg_km'] = int(result['lub_do_przebieg_km'][:-3].replace(' ', ''))
    if 'Miesięczna_rata' in result:
        result['Miesięczna_rata'] = int(result['Miesięczna_rata'][:-3].replace(' ', ''))
    if 'Opłata_początkowa' in result:
        result['Opłata_początkowa'] = int(result['Opłata_początkowa'][:-3].replace(' ', ''))
    if 'Liczba_pozostałych_rat' in result:
        result['Liczba_pozostałych_rat'] = int(result['Liczba_pozostałych_rat'])
    if 'Wartość_wykupu' in result:
        result['Wartość_wykupu'] = int(result['Wartość_wykupu'][:-3].replace(' ', ''))

    return result

SyntaxError: invalid syntax (Temp/ipykernel_2760/3262319396.py, line 21)

In [ ]:
def getIDFromOffer(soup):
    return {'offer_id': soup.find('span', attrs={'id': 'ad_id'}).text}

In [ ]:
def getSellerFromOffer(soup):
    sellerName = soup.find('h2', attrs={'class': 'seller-heading__seller-name'}).text.strip()
    sellerType = soup.find('section', attrs={'class': 'seller-features'}).find('article', attrs={'class': 'seller-features__feature'}).text.strip()

    return {'seller_name': sellerName, 'seller_type': sellerType}

In [ ]:
def getPriceFromOffer(soup):
    priceCurrency = soup.find('span', attrs={'class': 'offer-price__number'}).text
    price, currency = map(lambda x: x.strip(), priceCurrency.replace(' ', '', 2).split(' ', 1))
    forNegotiation = soup.find('span', attrs={'class': 'offer-price__details'}).text

    return {'price': price, 'currency': currency, 'offer_type': forNegotiation}


In [ ]:
def getDescFromOffer(soup):
    return {'description': soup.find('div', attrs={'class': 'offer-description__description'}).text.replace("'", "''")}

In [ ]:
def getEquipmentFromOffer(soup):
    result = { i: False for i in eqOptionsTable}
    try:
        offerEq = soup.find('div', attrs={'class': 'offer-features'}).text
    except AttributeError:
        return result

    for (i, j) in zip(eqOptions, eqOptionsTable):
        if i in offerEq:
            result[j] = True

    return result

In [ ]:
try:
    content = requests.get('https://www.otomoto.pl/oferta/renault-captur-mega-stan-super-okazja-ID6E06JQ.html#7ce9f71ee3').content
except:
    pass
soup = BeautifulSoup(content, 'html5lib')

print(getEquipmentFromOffer(soup))

{'ABS': False, 'CD': False, 'Centralny_zamek': False, 'Elektryczne_szyby_przednie': False, 'Elektrycznie_ustawiane_lusterka': False, 'Immobilizer': False, 'Poduszka_powietrzna_kierowcy': False, 'Poduszka_powietrzna_pasażera': False, 'Radio_fabryczne': False, 'Wspomaganie_kierownicy': False, 'Alarm': False, 'Alufelgi': False, 'ASR_kontrola_trakcji': False, 'Asystent_parkowania': False, 'Asystent_pasa_ruchu': False, 'Bluetooth': False, 'Czujnik_deszczu': False, 'Czujnik_martwego_pola': False, 'Czujnik_zmierzchu': False, 'Czujniki_parkowania_przednie': False, 'Czujniki_parkowania_tylne': False, 'Dach_panoramiczny': False, 'Elektrochromatyczne_lusterka_boczne': False, 'Elektrochromatyczne_lusterko_wsteczne': False, 'Elektryczne_szyby_tylne': False, 'Elektrycznie_ustawiane_fotele': False, 'ESP_stabilizacja_toru_jazdy': False, 'Gniazdo_AUX': False, 'Gniazdo_SD': False, 'Gniazdo_USB': False, 'Hak': False, 'HUD_wyświetlacz_przezierny': False, 'Isofix': False, 'Kamera_cofania': False, 'Klimatyz

In [ ]:
def dropTable(cursor):
    cursor.execute('DROP TABLE IF EXISTS info')
    cursor.execute('DROP TABLE IF EXISTS details')
    cursor.execute('DROP TABLE IF EXISTS equipment')
    cursor.execute('DROP TABLE IF EXISTS seller')
    cursor.execute('DROP TABLE IF EXISTS description')
    cursor.execute('DROP TABLE IF EXISTS price')
    cursor.execute('DROP TABLE IF EXISTS offers')

In [ ]:
def createTables(cursor):
    mainTable = '''CREATE TABLE offers (
        offer_id BIGINT,
        url VARCHAR(255) NOT NULL,
        PRIMARY KEY(offer_id)
        )'''
    mainInfoTable = '''CREATE TABLE details (
        offer_id BIGINT NOT NULL,
        Oferta_od VARCHAR(255),
        Kategoria VARCHAR(255) NOT NULL,
        Marka_pojazdu VARCHAR(255) NOT NULL,
        Model_pojazdu VARCHAR(255) NOT NULL,
        Rodzaj_paliwa VARCHAR(255) NOT NULL,
        Moc INT NOT NULL,
        Pojemność_skokowa INT NOT NULL,
        Liczba_drzwi INT NOT NULL,
        Skrzynia_biegów VARCHAR(255) NOT NULL,
        Wersja VARCHAR(255),
        Generacja VARCHAR(255),
        Segment VARCHAR(255),
        Kolor VARCHAR(255) NOT NULL,
        Przebieg INT NOT NULL,
        Rok_produkcji INT NOT NULL,
        Napęd VARCHAR(255),
        Spalanie_W_Mieście FLOAT,
        Typ_nadwozia VARCHAR(255),
        Emisja_CO2 INT,
        Liczba_miejsc INT,
        Rodzaj_koloru VARCHAR(255),
        Kraj_pochodzenia VARCHAR(255),
        Pierwszy_właściciel VARCHAR(255),
        Bezwypadkowy VARCHAR(255),
        Serwisowany_w_ASO VARCHAR(255),
        Stan VARCHAR(255),
        VAT_marża VARCHAR(255),
        Pierwsza_rejestracja VARCHAR(255),
        Numer_rejestracyjny_pojazdu VARCHAR(255),
        Zarejestrowany_w_Polsce VARCHAR(255),
        Filtr_cząstek_stałych VARCHAR(255),
        Tuning VARCHAR(255),
        Możliwość_finansowania VARCHAR(255),
        Uszkodzony VARCHAR(255),
        Faktura_VAT VARCHAR(255),
        Spalanie_W_Cyklu_Mieszanym FLOAT,
        Spalanie_Poza_Miastem FLOAT,
        Okres_gwarancji_producenta VARCHAR(255),
        Leasing VARCHAR(255),
        Miesięczna_rata INT,
        lub_do_przebieg_km INT,
        Gwarancja_dealerska_w_cenie VARCHAR(255),
        Homologacja_ciężarowa VARCHAR(255),
        Opłata_początkowa INT,
        Liczba_pozostałych_rat INT,
        Wartość_wykupu INT,
        Kierownica_po_prawej_Anglik VARCHAR(255),
        CONSTRAINT fk_details
            FOREIGN KEY(offer_id)
                REFERENCES offers(offer_id)
        )'''
    equipmentTable = '''CREATE TABLE equipment (
        offer_id BIGINT,
        ABS BOOLEAN,
        CD BOOLEAN,
        Centralny_zamek BOOLEAN,
        Elektryczne_szyby_przednie BOOLEAN,
        Elektrycznie_ustawiane_lusterka BOOLEAN,
        Immobilizer BOOLEAN,
        Poduszka_powietrzna_kierowcy BOOLEAN,
        Poduszka_powietrzna_pasażera BOOLEAN,
        Radio_fabryczne BOOLEAN,
        Wspomaganie_kierownicy BOOLEAN,
        Alarm BOOLEAN,
        Alufelgi BOOLEAN,
        ASR_kontrola_trakcji BOOLEAN,
        Asystent_parkowania BOOLEAN,
        Asystent_pasa_ruchu BOOLEAN,
        Bluetooth BOOLEAN,
        Czujnik_deszczu BOOLEAN,
        Czujnik_martwego_pola BOOLEAN,
        Czujnik_zmierzchu BOOLEAN,
        Czujniki_parkowania_przednie BOOLEAN,
        Czujniki_parkowania_tylne BOOLEAN,
        Dach_panoramiczny BOOLEAN,
        Elektrochromatyczne_lusterka_boczne BOOLEAN,
        Elektrochromatyczne_lusterko_wsteczne BOOLEAN,
        Elektryczne_szyby_tylne BOOLEAN,
        Elektrycznie_ustawiane_fotele BOOLEAN,
        ESP_stabilizacja_toru_jazdy BOOLEAN,
        Gniazdo_AUX BOOLEAN,
        Gniazdo_SD BOOLEAN,
        Gniazdo_USB BOOLEAN,
        Hak BOOLEAN,
        HUD_wyświetlacz_przezierny BOOLEAN,
        Isofix BOOLEAN,
        Kamera_cofania BOOLEAN,
        Klimatyzacja_automatyczna BOOLEAN,
        Klimatyzacja_czterostrefowa BOOLEAN,
        Klimatyzacja_dwustrefowa BOOLEAN,
        Klimatyzacja_manualna BOOLEAN,
        Komputer_pokładowy BOOLEAN,
        Kurtyny_powietrzne BOOLEAN,
        Łopatki_zmiany_biegów BOOLEAN,
        MP3 BOOLEAN,
        Nawigacja_GPS BOOLEAN,
        Odtwarzacz_DVD BOOLEAN,
        Ogranicznik_prędkości BOOLEAN,
        Ogrzewanie_postojowe BOOLEAN,
        Podgrzewana_przednia_szyba BOOLEAN,
        Podgrzewane_lusterka_boczne BOOLEAN,
        Podgrzewane_przednie_siedzenia BOOLEAN,
        Podgrzewane_tylne_siedzenia BOOLEAN,
        Poduszka_powietrzna_chroniąca_kolana BOOLEAN,
        Poduszki_boczne_przednie BOOLEAN,
        Poduszki_boczne_tylne BOOLEAN,
        Przyciemniane_szyby BOOLEAN,
        Radio_niefabryczne BOOLEAN,
        Regulowane_zawieszenie BOOLEAN,
        Relingi_dachowe BOOLEAN,
        System_Start_Stop BOOLEAN,
        Szyberdach BOOLEAN,
        Światła_do_jazdy_dziennej BOOLEAN,
        Światła_LED BOOLEAN,
        Światła_przeciwmgielne BOOLEAN,
        Światła_Xenonowe BOOLEAN,
        Tapicerka_skórzana BOOLEAN,
        Tapicerka_welurowa BOOLEAN,
        Tempomat BOOLEAN,
        Tempomat_aktywny BOOLEAN,
        Tuner_TV BOOLEAN,
        Wielofunkcyjna_kierownica BOOLEAN,
        Zmieniarka_CD BOOLEAN,
        CONSTRAINT fk_eq
            FOREIGN KEY(offer_id)
                REFERENCES offers(offer_id)
        )'''
    sellerTable = '''CREATE TABLE seller (
        seller_id INT GENERATED ALWAYS AS IDENTITY,
        offer_id BIGINT,
        seller_name VARCHAR(255),
        seller_type VARCHAR(255),
        Lokalizacja VARCHAR(255),
        Sprzedawca_od INT,
        CONSTRAINT fk_seller
            FOREIGN KEY(offer_id)
                REFERENCES offers(offer_id)
        )'''
    descriptionTable = '''CREATE TABLE description (
        offer_id BIGINT,
        Description VARCHAR,
        CONSTRAINT fk_desc
            FOREIGN KEY(offer_id)
                REFERENCES offers(offer_id)
        )'''
    priceTable =  '''CREATE TABLE price (
        offer_id BIGINT,
        price INT NOT NULL,
        currency VARCHAR(10),
        offer_type VARCHAR(255),
        Rozliczanie VARCHAR(255),
        CONSTRAINT fk_price
            FOREIGN KEY(offer_id)
                REFERENCES offers(offer_id)
        )'''

    cursor.execute(mainTable)
    cursor.execute(mainInfoTable)
    cursor.execute(equipmentTable)
    cursor.execute(sellerTable)
    cursor.execute(descriptionTable)
    cursor.execute(priceTable)

In [ ]:
def keysAsList(dict):
    return [x for x in dict]

In [ ]:
def insertData(cursor, tableName, data):
    insertDataCommand = f'INSERT INTO {tableName} ('

    for col in data:
        insertDataCommand += f'{col}, '
    insertDataCommand = insertDataCommand[:-2] + ') VALUES ('

    for value in data.values():
        if isinstance(value, int) or isinstance(value, float):
            insertDataCommand += f"{value}, "
        else:
            insertDataCommand += f"'{value}', "
    insertDataCommand = insertDataCommand[:-2] + ') ON CONFLICT DO NOTHING'

    cursor.execute(insertDataCommand)

In [ ]:
sql = psycopg2.connect(host='192.168.10.163', port='5433', database='otomoto-1', user='leon', password='biznes')
cur = sql.cursor()

dropTable(cur)

createTables(cur)
sql.commit()

In [ ]:
firstLaunch = True
columns = []

counter = 0
for i in range(1, 20):
    for offerUrl in getOffersUrlsFromPage(f'https://www.otomoto.pl/osobowe/?search%5Border%5D=created_at%3Adesc&page={i}'):
        try:
            content = requests.get(offerUrl).content
        except:
            continue
        soup = BeautifulSoup(content, 'html5lib')

        offerID = getIDFromOffer(soup)
        equip = getEquipmentFromOffer(soup)
        details = getDetailsFromOffer(soup)
        seller = getSellerFromOffer(soup)
        money = getPriceFromOffer(soup)
        desc = getDescFromOffer(soup)
        
        print(offerUrl)
        insertData(cur, 'offers', offerID | {'url': offerUrl})
        insertData(cur, 'details', offerID | details)
        insertData(cur, 'equipment', offerID | equip)
        insertData(cur, 'price', offerID | money)
        insertData(cur, 'seller', offerID | seller)
        insertData(cur, 'description', offerID | desc)
        
        clear_output(wait=True)
        print(counter)
        counter += 1
    
sql.commit()
cur.close()
sql.close()


167


KeyError: 'Moc'